In [8]:
import torch
from torch import nn
from torchvision import transforms
import torchvision
from torch.utils import data

In [14]:
"""
第⼀卷积层有6个输出通道，使⽤5 × 5卷积核和⼀个sigmoid激活函数
第一个汇聚层是平均汇聚层，大小为2 × 2
第二个卷积层有6个输入通道，16个输出通道，使⽤5 × 5卷积核和⼀个sigmoid激活函数
第二个汇聚层是平均汇聚层，大小为2 × 2
后面是三个全连接层
"""
LeNet = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [4]:
# 查看数据通过网络的每一层之后的数据形状
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in LeNet:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape: \t', X.shape)

Conv2d output shape: 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([1, 6, 14, 14])
Conv2d output shape: 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape: 	 torch.Size([1, 16, 5, 5])
Flatten output shape: 	 torch.Size([1, 400])
Linear output shape: 	 torch.Size([1, 120])
Sigmoid output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])


In [7]:
# 查看各层参数
for layer in LeNet:
    print(layer.__class__.__name__)
    print(*[(name, param.shape) for name, param in layer.named_parameters()])

Conv2d
('weight', torch.Size([6, 1, 5, 5])) ('bias', torch.Size([6]))
Sigmoid

AvgPool2d

Conv2d
('weight', torch.Size([16, 6, 5, 5])) ('bias', torch.Size([16]))
Sigmoid

AvgPool2d

Flatten

Linear
('weight', torch.Size([120, 400])) ('bias', torch.Size([120]))
Sigmoid

Linear
('weight', torch.Size([84, 120])) ('bias', torch.Size([84]))
Sigmoid

Linear
('weight', torch.Size([10, 84])) ('bias', torch.Size([10]))


In [9]:
def load_data_fashion_mnist(batch_size, resize=None):
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root="data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(root="data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=4),
            data.DataLoader(mnist_test, batch_size, shuffle=False, num_workers=4))


batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size)

In [11]:
class Accumulator:
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    # 将y_hat进行类型转换
    cmp = y_hat.type(y.dtype) == y
    # 返回预测正确的个数
    return float(cmp.type(y.dtype).sum())


def evaluate_accuracy(net, data_iter):
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [36]:
def train(net, train_iter, test_iter, num_epochs, lr, optimizer="sgd"):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)

    net.apply(init_weights)
    # optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    else:
        # adam
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        for i, (X, y) in enumerate(train_iter):
            optimizer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            # print("epoch:{},batch:{},train accuracy:{}".format(epoch,i,accuracy(y_hat, y)/len(y)))
        print("epoch:{},acc:{}".format(epoch, evaluate_accuracy(net, test_iter)))

In [17]:
lr, num_epochs = 0.9, 10
train(LeNet, train_iter, test_iter, num_epochs, lr)

epoch:0,acc:0.1
epoch:1,acc:0.5784
epoch:2,acc:0.6363
epoch:3,acc:0.6789
epoch:4,acc:0.7595
epoch:5,acc:0.757
epoch:6,acc:0.7938
epoch:7,acc:0.7774
epoch:8,acc:0.7718
epoch:9,acc:0.8121


In [34]:
# 调整LeNet
# 扩大卷积核大小
# 更换sigmoid为RELU
# 扩大通道数量
LeNet = nn.Sequential(
    nn.Conv2d(1, 10, kernel_size=7, padding=2), nn.ReLU(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(10, 20, kernel_size=7), nn.ReLU(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(180, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, 10)
)

In [35]:
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in LeNet:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape: \t', X.shape)

Conv2d output shape: 	 torch.Size([1, 10, 26, 26])
ReLU output shape: 	 torch.Size([1, 10, 26, 26])
AvgPool2d output shape: 	 torch.Size([1, 10, 13, 13])
Conv2d output shape: 	 torch.Size([1, 20, 7, 7])
ReLU output shape: 	 torch.Size([1, 20, 7, 7])
AvgPool2d output shape: 	 torch.Size([1, 20, 3, 3])
Flatten output shape: 	 torch.Size([1, 180])
Linear output shape: 	 torch.Size([1, 120])
ReLU output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
ReLU output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])


In [37]:
lr, num_epochs = 0.5, 10
train(LeNet, train_iter, test_iter, num_epochs, lr)

epoch:0,acc:0.7491
epoch:1,acc:0.7892
epoch:2,acc:0.8332
epoch:3,acc:0.7812
epoch:4,acc:0.8063
epoch:5,acc:0.8319
epoch:6,acc:0.8092
epoch:7,acc:0.8669
epoch:8,acc:0.8746
epoch:9,acc:0.8625
